In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments

from sklearn.metrics import f1_score

In [13]:
dataset = load_dataset("GonzaloValdenebro/MedicalQuestionAnsweringDataset")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'Question', 'Context', 'Topic', 'Answer'],
        num_rows: 16406
    })
})

In [5]:
dataset["train"]

Dataset({
    features: ['id', 'Question', 'Context', 'Topic', 'Answer'],
    num_rows: 16406
})

In [6]:
dataset["train"][0]

{'id': 1,
 'Question': 'What is (are) keratoderma with woolly hair ?',
 'Context': 'Keratoderma with woolly hair is a group of related conditions that affect the skin and hair and in many cases increase the risk of potentially life-threatening heart problems. People with these conditions have hair that is unusually coarse, dry, fine, and tightly curled. In some cases, the hair is also sparse. The woolly hair texture typically affects only scalp hair and is present from birth. Starting early in life, affected individuals also develop palmoplantar keratoderma, a condition that causes skin on the palms of the hands and the soles of the feet to become thick, scaly, and calloused.  Cardiomyopathy, which is a disease of the heart muscle, is a life-threatening health problem that can develop in people with keratoderma with woolly hair. Unlike the other features of this condition, signs and symptoms of cardiomyopathy may not appear until adolescence or later. Complications of cardiomyopathy ca

In [7]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [8]:
type(dataset["train"])

datasets.arrow_dataset.Dataset

In [9]:
dataset["train"][1]

{'id': 2,
 'Question': 'How many people are affected by keratoderma with woolly hair ?',
 'Context': 'Keratoderma with woolly hair is rare; its prevalence worldwide is unknown.  Type I (Naxos disease) was first described in families from the Greek island of Naxos. Since then, affected families have been found in other Greek islands, Turkey, and the Middle East. This form of the condition may affect up to 1 in 1,000 people from the Greek islands.  Type II (Carvajal syndrome), type III, and type IV have each been identified in only a small number of families worldwide.',
 'Topic': 'growth_hormone_receptor',
 'Answer': 'up to 1 in 1,000'}

In [10]:
len(dataset["train"])

16406

In [11]:
idx = 500




print(f"Question: {dataset['train'][idx]['Question']}")
print(f"Context: {dataset['train'][idx]['Context']}")
print(f"\nAnswer: {dataset['train'][idx]['Answer']}")

Question: What is (are) glucose-6-phosphate dehydrogenase deficiency ?
Context: Glucose-6-phosphate dehydrogenase deficiency is a genetic disorder that occurs most often in males. This condition mainly affects red blood cells, which carry oxygen from the lungs to tissues throughout the body. In affected individuals, a defect in an enzyme called glucose-6-phosphate dehydrogenase causes red blood cells to break down prematurely. This destruction of red blood cells is called hemolysis.  The most common medical problem associated with glucose-6-phosphate dehydrogenase deficiency is hemolytic anemia, which occurs when red blood cells are destroyed faster than the body can replace them. This type of anemia leads to paleness, yellowing of the skin and whites of the eyes (jaundice), dark urine, fatigue, shortness of breath, and a rapid heart rate. In people with glucose-6-dehydrogenase deficiency, hemolytic anemia is most often triggered by bacterial or viral infections or by certain drugs (su

In [12]:
# Flattening the datasets
flattened_train_data = dataset['train'].flatten()

In [13]:
flattened_train_data

Dataset({
    features: ['id', 'Question', 'Context', 'Topic', 'Answer'],
    num_rows: 16406
})

In [2]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer.model_max_length = 512

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
idx = 300

question = dataset["train"][idx]["Question"]
context = dataset['train'][idx]['Context']

NameError: name 'dataset' is not defined

In [ ]:
question

'What is (are) STING-associated vasculopathy with onset in infancy ?'

In [10]:
context

NameError: name 'context' is not defined

In [11]:
len(context)

NameError: name 'context' is not defined

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")

In [5]:
if not next(model.parameters()).is_cuda:
  model.to("cuda")

In [6]:
model.to("cpu")

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [9]:
text = """
John Doe, a 45-year-old male with a history of type 2 diabetes mellitus diagnosed five years ago, presents with concerning symptoms. 
He has been undergoing metformin therapy to manage his blood sugar levels. Recent blood tests reveal elevated HbA1c levels, 
indicating poor glycemic control. Mr. Doe reports experiencing frequent episodes of fatigue, increased thirst, and blurred vision. 
He denies any recent changes in his diet or exercise routine. Upon physical examination, Mr. Doe is found to be obese with a body mass index 
(BMI) of 32 kg/m². Neurological examination findings are unremarkable.


"""

questions = ["What symptoms does John Doe report, and what does his recent blood test indicate about his condition?",
             "what is the body mass index of John Doe?"]


In [10]:
for question in questions:
    inputs = tokenizer.encode_plus(question, text, return_tensors="pt")

    input_ids = inputs["input_ids"].tolist()[0]
#    inputs.to("cuda")
    inputs.to("cpu") # for ansh laptop
    
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_model = model(**inputs)
    
    start_logits = answer_model['start_logits'].cpu().detach().numpy()

    answer_start = np.argmax(start_logits)  
    
    end_logits = answer_model['end_logits'].cpu().detach().numpy()
    
    answer_end = np.argmax(end_logits) + 1 

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")


Question: What symptoms does John Doe report, and what does his recent blood test indicate about his condition?
Answer: fatigue, increased thirst, and blurred vision

Question: what is the body mass index of John Doe?
Answer: 32 kg / m²



In [6]:
import pickle

# Assuming 'model' is your trained model object
with open("QA_Model_cpu.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
conversation